# Data Cleaning

First we create a python conda environment to use for the rest of this project for reproducibility

In [1]:
!conda create -y -n wb-analysis-env -c conda-forge python=3.10 ipykernel=6.29.3 numpy=1.26.4 pandas=2.2.1 matplotlib=3.8.4 seaborn=0.13.2 wbgapi=1.0.12 requests=2.31.0 sqlalchemy=2.0.29
!python -m ipykernel install --user --name wb-analysis-env --display-name "Python (wb-analysis-env)"
!conda activate wb-analysis-env

Channels:
 - conda-forge
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /Users/Tom/miniforge3/envs/wb-analysis-env

  added / updated specs:
    - ipykernel=6.29.3
    - matplotlib=3.8.4
    - numpy=1.26.4
    - pandas=2.2.1
    - python=3.10
    - requests=2.31.0
    - seaborn=0.13.2
    - sqlalchemy=2.0.29
    - wbgapi=1.0.12


The following NEW packages will be INSTALLED:

  _openmp_mutex      conda-forge/osx-arm64::_openmp_mutex-4.5-7_kmp_llvm 
  appnope            conda-forge/noarch::appnope-0.1.4-pyhd8ed1ab_1 
  asttokens          conda-forge/noarch::asttokens-3.0.1-pyhd8ed1ab_0 
  backports.zstd     conda-forge/osx-arm64::backports.zstd-1.1.0-py310hdc7f11d_1 
  brotli             conda-forge/osx-arm64::brotli-1.2.0-h7d5ae5b_1 
  brotli-bin         conda-forge/osx-arm64::brotli-bin-1.2.0-hc919400_1 
  brotli-python      conda-forge/osx-arm64::brotli-python-1.2.0-py310h6123dab_1 
  bzip2              conda-forge/osx-arm64::bzip2-1.0.8-hd

### 1. Imports and Setup
We import `wbgapi` to fetch data directly from the World Bank and standard libraries for file handling.

In [2]:
import wbgapi as wb
import pandas as pd
import os
from sqlalchemy import create_engine, text

# Define file paths
DATA_DIR = '../data'
SCRIPTS_DIR = '../scripts'
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(SCRIPTS_DIR, exist_ok=True)

### 2. Data Retrieval
Here we define the indicators map and the specific countries we want to analyze. We then query the World Bank API for the years 2017-2023.

In [6]:
indicators = {
    'NY.GDP.PCAP.KD': 'gdp_pc',
    'SL.EMP.TOTL.SP.ZS': 'emp_ratio',
    'NY.GDP.MKTP.KD.ZG': 'gdp_growth',
    'SL.TLF.0714.ZS': 'child_labor_rate',
    'SE.PRM.UNER': 'children_out_of_school_primary'
}

countries = ['PRT', 'GBR', 'NOR', 'ALB', 'UKR', 'ITA']
years = range(2017, 2024)

raw_df = wb.data.DataFrame(indicators.keys(), economy=countries, time=years,
                           numericTimeKeys=True, labels=True, columns='series')

raw_df = raw_df.reset_index()
if 'Time' in raw_df.columns:
    raw_df = raw_df.drop(columns=['Time'])
raw_df.head()

,economy,time,Country,NY.GDP.MKTP.KD.ZG,NY.GDP.PCAP.KD,SE.PRM.UNER,SL.EMP.TOTL.SP.ZS,SL.TLF.0714.ZS
0,ITA,2023,Italy,0.715373,34146.023226,52117.0,46.002,NaN
1,ITA,2022,Italy,4.821177,33891.886653,64907.0,45.078,NaN
2,ITA,2021,Italy,8.931062,32267.709105,55640.0,43.842,NaN
3,ITA,2020,Italy,-8.868221,29469.798924,39912.0,43.832,NaN
4,ITA,2019,Italy,0.429163,32180.434102,43971.0,44.651,NaN


### 3. SQL Data Cleaning & Statistics
We use SQL to:
1. Clean: Select specific columns, rename them, and exclude empty columns.
2. Analyze: Calculate descriptive statistics.

In [ ]:
engine = create_engine('sqlite:///:memory:')
raw_df.to_sql('world_bank_raw', engine, index=False, if_exists='replace')

# Renamed all columns to understandandable names and removed child labor column as it appears to be all missing values
cleaning_query = """
SELECT 
    economy AS iso_code,
    Country AS country_name,
    CAST(time AS INTEGER) AS year,
    "NY.GDP.PCAP.KD" AS gdp_pc,
    "SL.EMP.TOTL.SP.ZS" AS emp_ratio,
    "NY.GDP.MKTP.KD.ZG" AS gdp_growth,
    "SE.PRM.UNER" AS children_out_of_school_primary
FROM world_bank_raw
ORDER BY economy, time;
"""

stats_query = """
SELECT 
    Country AS country_name,
    ROUND(AVG("NY.GDP.PCAP.KD"), 2) AS avg_gdp_pc,
    ROUND(MAX("NY.GDP.MKTP.KD.ZG"), 2) AS max_gdp_growth
FROM world_bank_raw
GROUP BY Country
ORDER BY avg_gdp_pc DESC;
"""

df_clean = pd.read_sql(cleaning_query, engine)
df_stats = pd.read_sql(stats_query, engine)

sql_file_path = os.path.join(SCRIPTS_DIR, 'data_processing.sql')
with open(sql_file_path, "w") as f:
    f.write(f"/* DATA CLEANING QUERY */\n{cleaning_query}\n\n")
    f.write(f"/* DESCRIPTIVE STATS QUERY */\n{stats_query}\n")

display(df_stats)

,country_name,avg_gdp_pc,max_gdp_growth
0,Norway,77261.33,3.91
1,United Kingdom,46278.41,8.58
2,Italy,32201.54,8.93
3,Portugal,21115.96,6.99
4,Albania,4748.84,8.97
5,Ukraine,2244.88,5.53


### Saving Data
Finally, we reorder the columns to put identifiers first and save the clean dataset to the `../data/` directory.

In [9]:
output_path = os.path.join(DATA_DIR, 'cleaned_wb_data.csv')
df_clean.to_csv(output_path, index=False)